In [ ]:
import pickle as pickle
import numpy as np
base_path="../data/data_m3/"
with open(base_path+'binary_train_codes_x.pkl', 'rb') as f0:
  binary_train_codes_x = pickle.load(f0)

with open(base_path+'binary_test_codes_x.pkl', 'rb') as f1:
  binary_test_codes_x = pickle.load(f1)

train_codes_y = np.load(base_path+'train_codes_y.npy')
train_visit_lens = np.load(base_path+'train_visit_lens.npy')

test_codes_y = np.load(base_path+'test_codes_y.npy')
test_visit_lens = np.load(base_path+'test_visit_lens.npy')
train_pids = np.load(base_path+'train_pids.npy')

test_pids = np.load(base_path+'test_pids.npy')
with open(base_path+'patient_time_duration_encoded.pkl', 'rb') as f80:
  patient_time_duration_encoded = pickle.load(f80)

In [ ]:
import numpy as np
import pickle
with open("tma_result_m3/m3_tma_r1.pkl", "rb") as f:
    data = pickle.load(f)


In [ ]:
import json
with open("cra_result_m3/m3_cra_r1.json", "r",encoding="utf-8") as f:
    hist_llm_result=json.load(f)   

In [ ]:
import numpy as np
import pickle

with open(base_path+"code_map.pkl", "rb") as f:
    code_map = pickle.load(f)

inv_code_map = {v-1: k for k, v in code_map.items()}
print(code_map)
def fun1(idx):
    result=[]
    for id in idx:
        result.append(inv_code_map[id])
    return result

In [ ]:
import pandas as pd
df = pd.read_csv(base_path+'diagnosis_icd9_ontology.csv')
code2name = dict(zip(df['code'], df['name']))

In [ ]:
import pickle

with open(base_path+"code_map.pkl", "rb") as f:
    code_map = pickle.load(f)

inv_code_map = {v-1: k for k, v in code_map.items()}

code_map_idx=list(code_map.keys())
import pandas as pd


df = pd.read_csv(base_path+'diagnosis_icd9_ontology.csv')
df['code'] = df['code'].astype(str).str.strip()
df['name'] = df['name'].astype(str).str.strip()
df_f = df[df['code'].isin(code_map_idx)].copy()
name2code = dict(zip(df_f['name'].str.lower(), df_f['code']))

In [ ]:
def extract_diagnoses_final(pred_str: str, name_to_code: dict):
    diagnoses_with_pos = []
    pred_str_lower = pred_str.lower()

    for name in name_to_code:
        name_lower = name.lower()
        index = pred_str_lower.find(name_lower)
        if index != -1:
            diagnoses_with_pos.append((index, name_lower))
    diagnoses_with_pos.sort(key=lambda x: x[0])
    return [name for _, name in diagnoses_with_pos]

In [ ]:
def remove_duplicates_keep_order(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

In [ ]:
def get_diag_str(diag_list):
    temp_str_list=[]
    for diag in diag_list:
        temp_str_list.append(f'"{code2name[diag]}"')
    temp_str=', '.join(temp_str_list)
    return temp_str

In [ ]:
import inflect

def number_to_capitalized_ordinal(n):
    p = inflect.engine()
    return p.ordinal(p.number_to_words(n)).capitalize()

# 示例
print(number_to_capitalized_ordinal(1))
print(number_to_capitalized_ordinal(42)) 


In [ ]:
results=[]
all_ja=[]
ode_idx=0
top_k=250
for i in range(len(test_codes_y)):
    if test_visit_lens[i]<2:
        continue
    pids=test_pids[i]
    hist_diag=binary_test_codes_x[i]
    target_diag=test_codes_y[i]
    hist_diag_code=[]

    ode_pred=data[ode_idx]
    
    ode_pred = np.argsort(-ode_pred, axis=-1)
    
    ode_pred_code=[]

        
    ode_pred_code.append(list(fun1(ode_pred[:top_k])))
    for v in hist_diag:
        hist_diag_code.append(fun1(np.where(v == 1)[0]))
    instruction="You are an experienced medical diagnosis expert specializing in longitudinal disease progression and comorbidity reasoning. Your goal is to re-rank the candidate diseases for the patient’s final visit through a structured debate process between clinical reasoning and model prediction, leading to a clinically consistent final answer.\nYou will compare:\n1.Clinically Reasoned List – derived from disease evolution and comorbidity logic based on past visits.\n2.Model-Predicted List – derived from automated analysis of disease patterns.\n\nThen, perform a debate-style reconciliation: \n- Evaluate overlaps, inconsistencies, and missing diseases.\n- Prioritize diseases that are clinically progressive, pathophysiologically linked, or historically recurrent.\n- Down-rank transient, inconsistent, or unlikely conditions."

    duration=patient_time_duration_encoded[pids]
    
    input_="Patient History\n"
    for idx,code_list in enumerate(hist_diag_code):
        if idx==0:
            cur_duration=0
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit:"
        else:
            cur_duration+=duration[idx]
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit: ({cur_duration} days later):"

        temp+=f"\n- Diagnoses："+"{"+get_diag_str(code_list)+"}"
        input_+=temp+"\n\n"
        
    pred_str=hist_llm_result[ode_idx]['pred'].lower()
    pred_str=pred_str.replace("*","")
    diagnoses=extract_diagnoses_final(pred_str.lower(),name2code)

    codes = []
    for name in diagnoses:
        code = name2code.get(name)
        if code is not None:
            codes.append(code)

    pred_list=codes
    pred_list = remove_duplicates_keep_order(pred_list)
    temp=f"Two Candidate Lists for the Final Visit ({sum(duration)} days later):\n"
    temp+="Clinically Reasoned List："+f"{get_diag_str(pred_list)}\n\n"
    temp+="Model-Predicted List："+f"{get_diag_str(ode_pred_code[-1][:20])}\n"


    temp+=f"\n\nYour task:\n- Compare both lists and identify diseases supported by clinical progression patterns.\nOutput the final re-ranked list of diseases for the final visit in descending order of likelihood, integrating clinical reasoning and model evidence."+"\nDirectly provide the reordered list of disease names in descending order of likelihood. \nOutput format:\nAnswer: <Disease 1>, <Disease 2>, <Disease 3>, ..."
    input_+=temp+"\n"
    result={
        "pid":pids,
        "hist_diag_code":hist_diag_code,
        "target_diag_code":fun1(np.where(target_diag == 1)[0]),
        "instruction":instruction,
        "input":input_
        
    }
    ode_idx+=1
    results.append(result)

In [ ]:
print(results[0]['instruction'] + '\n' + results[0]['input'])


In [ ]:
len(results)

In [ ]:
import json
def np_encoder(obj):
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()         
    if isinstance(obj, np.ndarray):
        return obj.tolist()        
    return str(obj)              

with open(f"col_prompt_m3/m3_col_r1.json", "w",
          encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False,default=np_encoder, indent=2)